<a href="https://colab.research.google.com/github/rediff-pro/pytorch-learning/blob/dev/Tensorflow_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
if torch.cuda.is_available():
  print('We have GPU')

We have GPU


In [ ]:
if torch.cuda.is_available():
  gpu_rand = torch.rand(2, 2, device='cuda')
  print(gpu_rand)
else:
  print('Sorry, CPU only')

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# !wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
# !unzip ml-latest-small.zip

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
PATH = Path('ml-latest-small')
list(PATH.iterdir())

In [ ]:
! head $PATH/ratings.csv

In [ ]:
data = pd.read_csv(PATH/"ratings.csv")
data.head()

In [ ]:
# spliting the data to training and validation set
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()


In [ ]:
# len(data)
# np.random.rand(len(data))

In [ ]:
def proc_col(col, train_col=None):
  if train_col is not None:
    uniq = train_col.unique()
  else:
    uniq = col.unique()
  name2idx = {o:i for i,o in enumerate(uniq)}
  return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [ ]:
def encode_data(df, train=None):
  df = df.copy()
  for col_name in ["userId", "movieId"]:
    train_col = None
    if train is not None:
      train_col = train[col_name]
    _, col, _ = proc_col(df[col_name], train_col)
    df[col_name] = col
    df = df[df[col_name] >= 0]
  return df

In [ ]:
df_t = pd.read_csv('https://raw.githubusercontent.com/yanneta/pytorch-tutorials/refs/heads/master/images/tiny_training2.csv')
df_v = pd.read_csv('https://raw.githubusercontent.com/yanneta/pytorch-tutorials/refs/heads/master/images/tiny_val2.csv')
print(df_t)
df_t_e = encode_data(df_t)
df_v_e = encode_data(df_v, df_t)
df_v_e

In [ ]:
# encoding training and validation data
df_train = encode_data(train)
df_val = encode_data(val, train)


#Embedding Layer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
embed = nn.Embedding(10, 3)
embed

In [ ]:
a = torch.LongTensor([[1,2,0,4,5,1]])
embed(a)

## Matix factorization Model

In [ ]:
class MF(nn.Module):
  def __init__(self, num_users, num_items, emb_size=100):
    super(MF, self).__init__()
    self.user_emb = nn.Embedding(num_users, emb_size)
    self.item_emb = nn.Embedding(num_items, emb_size)
    self.user_emb.weight.data.uniform_(0, 0.05)
    self.item_emb.weight.data.uniform_(0, 0.05)

  def forward(self, u, v):
    u = self.user_emb(u)
    v = self.item_emb(v)
    return (u*v).sum(1)


In [ ]:
df_t_e

In [ ]:
num_users = 7
num_items = 7
emb_size = 3
user_emb = nn.Embedding(num_users, emb_size)
item_emb = nn.Embedding(num_items, emb_size)
users = torch.LongTensor(df_t_e.userId.values)
items =torch.LongTensor(df_t_e.movieId.values)


In [ ]:
U = user_emb(users)
V = item_emb(items)

In [ ]:
U

In [ ]:
U*V

tensor([[-0.0095, -0.1877, -0.0776],
        [ 0.0288,  0.3213,  0.2104],
        [-0.5717, -0.0184,  1.2763],
        [ 0.1099,  0.0034,  0.3706],
        [ 0.4936, -0.8570,  0.2633],
        [-1.4894,  1.4674, -0.7140],
        [-0.2500,  0.0293,  0.7439],
        [ 0.6399,  0.0154, -2.0605],
        [-0.0651, -0.9373,  0.5122],
        [ 0.1667, -0.4910, -1.4188],
        [ 1.3837, -0.5532, -1.0287],
        [-1.7094,  0.5366, -1.0676],
        [-1.4499, -0.1642, -1.0906]], grad_fn=<MulBackward0>)

In [ ]:
(U*V).sum(1)

tensor([-0.2748,  0.5605,  0.6862,  0.4840, -0.1001, -0.7361,  0.5232, -1.4052,
        -0.4902, -1.7431, -0.1981, -2.2404, -2.7047], grad_fn=<SumBackward1>)

## Training MF model

In [ ]:
num_users = len(df_train.userId.unique())
num_items = len(df_train.movieId.unique())
print(num_users, num_items)

610 8998


In [ ]:
model = MF(num_users, num_items, emb_size).cuda(device='cuda')

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def train_epocs(mode, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
  model.train()
  for i in range(epochs):
    users = torch.LongTensor(df_train.userId.values).cuda(device='cuda')
    items = torch.LongTensor(df_train.movieId.values).cuda(device='cuda')
    ratings = torch.FloatTensor(df_train.rating.values).cuda(device='cuda')
    if unsqueeze:
      ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())
  test_loss(model, unsqueeze)

In [ ]:
# Here is what unsqueese does
ratings = torch.FloatTensor(df_train.rating.values)
print(ratings.shape)
ratings = ratings.unsqueeze(1).cuda(device='cuda')
print(ratings.shape)

In [ ]:
def test_loss(mode, unsqueese=False):
  mode.eval()
  users = torch.LongTensor(df_val.userId.values).cuda(device='cuda')
  items = torch.LongTensor(df_val.movieId.values).cuda(device='cuda')
  ratings = torch.FloatTensor(df_val.rating.values).cuda(device='cuda')
  if unsqueese:
    ratings = ratings.unsqueeze(1)
  y_hat = model(users, items)
  loss = F.mse_loss(y_hat, ratings)
  print('test loss %.3f'%loss.item())

In [ ]:
train_epocs(model,epochs=10, lr=0.1)

In [ ]:
train_epocs(model,epochs=15, lr=0.01)

In [ ]:
train_epocs(model,epochs=15, lr=0.01)

### MF with bias

In [ ]:
class MF_bias(nn.Module):
  def __init__(self,num_users, num_items, emb_size=100):
    super(MF_bias, self).__init__()
    self.user_emb = nn.Embedding(num_users, emb_size)
    self.user_bias = nn.Embedding(num_users,1)
    self.item_emb = nn.Embedding(num_items, emb_size)
    self.item_bias = nn.Embedding(num_items, 1)
    self.user_emb.weight.data.uniform_(0, 0.05)
    self.item_emb.weight.data.uniform_(0, 0.05)
    self.user_bias.weight.data.uniform_(-0.01, 0.01)
    self.item_bias.weight.data.uniform_(-0.01, 0.01)

  def forward(self, u, v):
    U = self.user_emb(u)
    V = self.item_emb(v)
    b_u = self.user_bias(u).squeeze()
    b_v = self.item_bias(v).squeeze()
    return (U*V).sum(1) + b_u * b_v




In [ ]:
model = MF_bias(num_users, num_items, emb_size=100).cuda(device='cuda')

In [ ]:
train_epocs(model, epochs=10, lr=0.05, wd=1e-5)

In [ ]:
train_epocs(model, epochs=10, lr=0.01, wd=1e-5)

In [ ]:
train_epocs(model, epochs=10, lr=0.001, wd=1e-5)

## Neural network model

In [ ]:
class CollabFNet(nn.Module):
  def __init__(self, num_users, num_items, emb_size=100, n_hidden=10):
    super(CollabFNet,self).__init__()
    self.user_emb = nn.Embedding(num_users, emb_size)
    self.item_emb = nn.Embedding(num_items, emb_size)
    self.lin1 = nn.Linear(emb_size*2, n_hidden)
    self.lin2 = nn.Linear(n_hidden, 1)
    self.drop1 = nn.Dropout(0.1)
  def forward(self, u, v):
    U = self.user_emb(u)
    V = self.user_emb(v)
    x = F.relu(torch.cat([U,V],dim=1))
    x = self.drop1(x)
    x = F.relu(self.lin1(x))
    x = self.lin2(x)
    return x

In [ ]:
model = CollabFNet(num_users, num_items, emb_size=100).cuda(device='cuda')

In [ ]:
train_epocs(model, epochs=15, lr=0.05, wd=1e-6, unsqueeze=True).cuda(device='cuda')